In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import os
import json
import time
# Census API Key
from config import api_key

In [2]:
#census_data = c.acs5.get().json()
query_url = f"https://api.census.gov/data/2019/acs/acs5"
display(query_url)
#https://api.census.gov/data#/2019/acs/acs5&{api_key}#?get=NAME,group(B01001)&for=us:1&key={api_key}

'https://api.census.gov/data/2019/acs/acs5'

In [3]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

cols = ["Zipcode", "Household Income", "Income per Capita", "Population",
        "Median Contract Rent", "Median Gross Rent", "Median Home Value",
        "Median Monthly Owner Costs","Median Age", "Year"]

df = pd.DataFrame(columns = cols)

years = [2019]

for year in years:

    c = Census(api_key, year=year)
    census_data = c.acs5.get(("NAME", "B19013_001E", 
                              "B19301_001E", 
                              "B01003_001E",
                              "B25058_001E",
                              "B25064_001E", 
                              "B25077_001E", 
                              "B25088_002E", 
                              "B01002_001E"), {'for': 'zip code tabulation area:*'})

    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column Reordering
    census_pd = census_pd.rename(columns={"B19013_001E": "Household Income",
                                          "B19301_001E": "Income per Capita",
                                          "B01003_001E": "Population",
                                          "B25058_001E": "Median Contract Rent",
                                          "B25064_001E": "Median Gross Rent",
                                          "B25077_001E": "Median Home Value",
                                          "B25088_002E": "Median Monthly Owner Costs",
                                          "B01002_001E": "Median Age",
                                          "NAME": "Name", "zip code tabulation area": "Zipcode"})


    # Final DataFrame
    census_pd['Year']=str(year)
    
    census_pd = census_pd[cols]
    
    
    concat_df = pd.concat([df, census_pd])

# Visualize
#print(len(census_pd))
display(concat_df.head())

CensusException: Sorry, the system is currently undergoing maintenance or is busy.  Please try again later.

In [ ]:
#Need to clean or drop the data with 666666 in it.

In [ ]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
concat_df.to_csv("census_data.csv", encoding="utf-8", index=False)

In [ ]:
!ls

In [ ]:
#create DataFrame for MSA
#https://www.roelpeters.be/solved-dtypewarning-columns-have-mixed-types-specify-dtype-option-on-import-or-set-low-memory-in-pandas/
msa_df = pd.read_csv('../project_1/files/ScanUSZipCode2017A.csv',low_memory=False)
msa_df

In [ ]:
#Comparing Data Types between the two diffrent dataframes to make certain they match for the merge
print(msa_df.dtypes)

In [ ]:
print(concat_df.dtypes) 
    

In [ ]:
#convert Zipcode datatype in concat_df to be int64
#https://www.kite.com/python/answers/how-to-convert-a-pandas-dataframe-column-from-object-to-int-in-python
concat_df["Zipcode"] = concat_df["Zipcode"].astype(object).astype(int)

In [ ]:
#Rename ZIP column in MSA to match Zipcode from Census data
#https://note.nkmk.me/en/python-pandas-dataframe-rename/
msa_df.rename(columns={'ZIP': 'Zipcode'}, inplace=True)


In [ ]:
#Merge data frames and drop the values in the census data with -666666666
merged_census_df = pd.merge(concat_df, msa_df, how="left", on=["Zipcode", "Zipcode"])
merged_census_df
merged_census_df.drop(merged_census_df[merged_census_df["Household Income"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Income per Capita"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Population"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Contract Rent"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Gross Rent"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Home Value"] == -666666666].index, inplace = True)       
merged_census_df.drop(merged_census_df[merged_census_df["Median Monthly Owner Costs"] == -666666666].index, inplace = True)      
merged_census_df.drop(merged_census_df[merged_census_df["Median Age"] == -666666666].index, inplace = True) 
       

In [ ]:
# Save as a csv to check full data set
# Note to avoid any issues later, use encoding="utf-8"
merged_census_df.to_csv("census_data_2019Years.csv", encoding="utf-8", index=False)